# request a chiaracer



In [11]:
import chi
# Before we go any further, we need to select which Chameleon site we will be using.
chi.use_site("CHI@Edge")
chi.set("project_name", "CHI-220971")
from chi import container
from chi import lease

Now using CHI@Edge:
URL: https://chi.edge.chameleoncloud.org
Location: University of Chicago, Chicago, Illinois, USA
Support contact: help@chameleoncloud.org


In [12]:
# from blazarclient.exception import BlazarClientException

lease_name = "chiaracer-car-lease"

start, end = lease.lease_duration(days=2)
reservations = []

try:
    container_lease = lease.get_lease(lease_name)
except ValueError:
    lease.add_device_reservation(reservations, device_name="chiaracer-2", count=1)
    container_lease = lease.create_lease(lease_name, reservations)
    lease_id = container_lease["id"]
    print(f"Requested new lease {container_lease['name']} with id: {lease_id}")
else:
    lease_id = container_lease.get("id")
    print(f"Reusing existing lease {container_lease['name']} with id: {lease_id}")
# Wait for the lease before proceeding
print("Waiting for lease to start ...")
lease.wait_for_active(lease_id)
print("Done!")

Reusing existing lease chiaracer-car-lease with id: 25ae3777-5d37-40dc-8cee-5d5595ffac83
Waiting for lease to start ...
Done!


In [ ]:

container_lease = lease.get_lease("chiaracer-car-lease")


lease_id = container_lease["id"]
print(f"lease_id: {lease_id}")


In [46]:
# Decide what container you want to use: main or specific hash

print("Requesting container ...")

container_name = "chiaracer-container"

my_container = container.create_container(
    container_name,
    #image="rianders/car:b529fe8",
    image="rianders/car:main",
    exposed_ports=["22"],
    interactive=True,
    command=["/bin/bash"],
    device_profiles=[
        "pi_camera",
        "pi_gpio",
        "pi_serial"],
    reservation_id=lease.get_device_reservation(lease_id),
)

print(f"Status is: {my_container.status}; with reason: {my_container.status_reason}")

print("Waiting for container to start ...")
container.wait_for_active(my_container.uuid)


Requesting container ...
Status is: Running; with reason: None
Waiting for container to start ...


<Container {'uuid': 'bcc4cd5a-cc6a-4a1c-b6bf-afd5dc867143', 'links': [{'href': 'http://chi.edge.chameleoncloud.org:9517/v1/containers/bcc4cd5a-cc6a-4a1c-b6bf-afd5dc867143', 'rel': 'self'}, {'href': 'http://chi.edge.chameleoncloud.org:9517/containers/bcc4cd5a-cc6a-4a1c-b6bf-afd5dc867143', 'rel': 'bookmark'}], 'name': 'chiaracer-container', 'project_id': '3be28ea43254412fb0d78d961d64637a', 'user_id': 'e03b99dada69c925afc2b7df92395a048f08b6ecd5484bb69a400e9056cd05e7', 'image': 'rianders/car:main', 'cpu': 0.0, 'cpu_policy': 'shared', 'memory': '0', 'command': [], 'status': 'Running', 'status_reason': None, 'task_state': None, 'environment': {}, 'workdir': None, 'auto_remove': True, 'ports': [22], 'hostname': None, 'labels': {}, 'addresses': {'db32fc77-deaa-4288-a480-17da94bbd5a1': [{'addr': '192.168.72.155', 'version': 4, 'port': '8a706d94-97cd-435c-8b6b-50ab35000ed7'}]}, 'restart_policy': {}, 'status_detail': None, 'interactive': True, 'tty': True, 'image_driver': 'docker', 'security_grou

In [ ]:
container_logs = container.get_logs(my_container.uuid)
print(container_logs)


In [ ]:
print(f"Status is: {my_container.status}; with reason: {my_container.status_reason}")

In [47]:
public_ip = container.associate_floating_ip(my_container.uuid)
print(f"public_ip: {public_ip}")
      

public_ip: 129.114.34.174


In [ ]:
# lease device list: https://chi.edge.chameleoncloud.org/project/leases/calendar/device/

In [ ]:
# Uncomment to destroy container
#container.destroy_container(my_container.uuid)

In [ ]:
# Remove lease when you're done with the device
#lease.delete_lease(lease_id)

In [48]:
cmd = "ls  -l"
print(cmd)

print(container.execute(my_container.uuid, cmd)["output"])

ls  -l
total 44
-rw-r--r-- 1 root root     0 May  1 02:03 README.rst
drwxr-xr-x 2 root root  4096 May  1 02:03 carservices
-rw-r--r-- 1 root root 13912 May  1 02:03 poetry.lock
-rw-r--r-- 1 root root   618 May  1 02:03 pyproject.toml
-rw-r--r-- 1 root root  8415 May  1 02:03 requirements.txt
-rw-r--r-- 1 root root   406 May  1 02:03 systeminstall.sh
drwxr-xr-x 2 root root  4096 May  1 02:03 tests



In [36]:
cmd =  "bash -c \"test_pihat\" > log.out"
print(container.execute(my_container.uuid,cmd)["output"])


/foocars/cars/carservices/carservices/pihattest.py:51: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(led, GPIO.OUT)



In [19]:
cmd = "ls"
print(container.execute(my_container.uuid, cmd)["output"])

README.rst
carservices
poetry.lock
pyproject.toml
requirements.txt
systeminstall.sh
tests



In [37]:
cmd =  "bash -c \"car_runner\" > log.out"
print(cmd)
print(container.execute(my_container.uuid,cmd)["output"])

In [39]:
cmd = "ls  ../chiaracer/"
print(cmd)
print(container.execute(my_container.uuid, cmd)["output"])

ls  ../chiaracer/
arduino
config.toml
data
readme.md



In [40]:
cmd = "ls  ../chiaracer/data/collected"
print(cmd)
print(container.execute(my_container.uuid, cmd)["output"])

ls  ../chiaracer/data/collected
readme.md

